In [1]:
import torch, os
from pytorch_pretrained_bert import OpenAIGPTTokenizer, OpenAIGPTLMHeadModel
from tqdm import tqdm
import numpy as np
from pytorch_pretrained_bert.file_utils import PYTORCH_PRETRAINED_BERT_CACHE
from pytorch_pretrained_bert.modeling import BertForSequenceClassification, BertConfig, WEIGHTS_NAME, CONFIG_NAME
#from pytorch_pretrained_bert.tokenization import BertTokenizer
from pytorch_pretrained_bert.optimization import BertAdam, warmup_linear
from bertviz.bertviz.pytorch_pretrained_bert import BertModel, BertTokenizer

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.
Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [3]:
# TODO consider is installing apex worth it??   ( who is using it?  is it pytorch_pretrained <-- which is huggingface)
# Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex
# using remote ssh 

# https://medium.com/the-artificial-impostor/use-nvidia-apex-for-easy-mixed-precision-training-in-pytorch-46841c6eed8c

In [2]:
#check if CUDA enabled
torch.cuda.is_available()  # The output should be a boolean "True"

True

## Delete and Generate ( B-GST )

In [2]:
special_tokens = ['<POS>', '<NEG>','<CON_START>','<START>','<END>'] # Set the special tokens
tokenizer = OpenAIGPTTokenizer.from_pretrained('openai-gpt', special_tokens=special_tokens)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = OpenAIGPTLMHeadModel.from_pretrained('openai-gpt', num_special_tokens=len(special_tokens))

torch.cuda.set_device(0)

ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.


In [3]:
## 1. Set Model Path to Downloaded Weights
#model_path = "./log_yelp_bert_best_head_preprocessed/pytorch_model_zero_grad_1.bin"  

model_path = "./emnlp_weights/B-GST/pytorch_model_zero_grad_1.bin"  #for YELP
model_path = "./data/imagecaption/bgst_classifier/pytorch_model_zero_grad_1.bin"
model_path = "./data/amazon/bgst_classifier/pytorch_model_zero_grad_1.bin"   #HERE  <--- 

path = os.path.join(os.getcwd(), model_path) 
model_state_dict = torch.load(path, map_location=device)
model.load_state_dict(model_state_dict)
model.to(device)
model.eval()

OpenAIGPTLMHeadModel(
  (transformer): OpenAIGPTModel(
    (tokens_embed): Embedding(40483, 768)
    (positions_embed): Embedding(512, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_1): BertLayerNorm()
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): BertLayerNorm()
      )
      (1): Block(
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_1): BertLayerNorm()
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout

In [5]:
!cat data/yelp/bert_classifier_2epochs/eval_results.txt

eval_accuracy = 0.982
eval_loss = 0.049413899958133695
global_step = 27704
loss = 0.03469694435766971


In [4]:
# 2. First do 1a and 1b from notes ( ie, data prep and then call run_classifier.py for yelp )
#    OR
#    try without training bert classifier at all ( which will give shitty results probably ?)
#     ie,  model = BertForSequenceClassification.from_pretrained(args.bert_model, num_labels=num_labels)
#          where args.bert_model = bert-base-uncased


#bert_classifier_dir = "./bert_classifier/" 
#bert_classifier_dir = "./data/yelp/bert_classifier_2epochs/"   #gives 98% acc  #for yelp

#bert_classifier_dir = "./data/imagecaption/bert_classifier_10epochs/"  #for imagecaption

bert_classifier_dir = "./data/amazon/bert_classifier_3epochs/"

model_cls = BertForSequenceClassification.from_pretrained(bert_classifier_dir, num_labels=2)
tokenizer_cls = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
model_cls.to(device)
model_cls.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
    

In [5]:
max_seq_len=70
sm = torch.nn.Softmax(dim=-1)

In [6]:
model.config.n_positions

512

In [7]:
def preditction_with_beam_search(ref_text, beam_width=3, vocab_length=40483):
    """
    This function decodes sentences using Beam Seach. 
    It will output #sentences = beam_width. This function works on a single example.
    
    ref_text : string : Input sentence
    beam_width : int : Width of the output beam
    vocab_length : int : Size of the Vocab after adding the special tokens
    """
    
    done = [False for i in range(beam_width)] # To track which beams are already decoded
    stop_decode = False
    decoded_sentences=[] # List of decoded sentences at any given time
    
    sm = torch.nn.Softmax(dim=-1) # To calculate Softmax over the final layer Logits
    tokens = tokenizer.tokenize(ref_text) # Tokenize the input text
    
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokens) # Convert tokens to ids
    index_tokens = [indexed_tokens for i in range(beam_width)] # Replication of Input ids for all the beams

    torch_tensor = torch.tensor(index_tokens).to(device)
    beam_indexes = [[] for i in range(beam_width)] # indexes of the current decoded beams
    best_scoes = [0 for i in range(beam_width)] # A list of lists to store Probability values of each decoded token of best beams
    count = 0
    while count < model.config.n_positions and not stop_decode:
        if count == 0: # For the first step when only one sentence is availabe
            with torch.no_grad():
                # Calculate output probability distribution over the Vocab,
                preds = sm(model(torch_tensor)) #  shape = [beam_bidth, len(input_sen)+1,Vocab_length]
            top_v, top_i = preds[:,-1,:].topk(beam_width) # Fatch top indexes and it's values
            [beam_indexes[i].append(top_i[0][i].tolist()) for i in range(beam_width)] # Update the Beam indexes
            # Update the best_scores, for first time just add the topk values directly
            for i in range(beam_width):
                best_scoes[i] = top_v[0][i].item()
            count += 1
        else: # After first step
            # Prepare the current_state by concating original input and decoded beam indexes
            current_state = torch.cat((torch_tensor, torch.tensor(beam_indexes).to(device)), dim=1)
            # Prediction on the current state
            with torch.no_grad():
                preds = sm(model(current_state))
            # Multiply new probability predictions with corresponding best scores
            # Total socres = beam_width * Vocab_Size
            flatten_score = (preds[:,-1,:]*torch.tensor(best_scoes).to(device).unsqueeze(1)).view(-1)
            # Fatch the top scores and indexes 
            vals, inx = flatten_score.topk(beam_width)
            # best_score_inx saves the index of best beams after multiplying the probability of new prediction
            best_scoes_inx = (inx / vocab_length).tolist()
            best_scoes = vals.tolist()
            # Unflatten the index 
            correct_inx = (inx % vocab_length).tolist()
            
            # Check if done for all the Beams
            for i in range(beam_width):
                if correct_inx[i] == tokenizer.special_tokens["<END>"]:
                    done[i] = True
            # Update the best score for each the current Beams
            for i in range(beam_width):
                if not done[i]:
                    best_scoes[i] = vals.tolist()[i]
            # Check is All the Beams are Done
            if (sum(done) == beam_width):
                stop_decode = True
            # Prepapre the new beams
            temp_lt=[0 for i in range(beam_width)]
            for i,x in enumerate(best_scoes_inx):
                temp_lt[i] = beam_indexes[x] + [correct_inx[i]]
            # Update the Beam indexes
            beam_indexes = temp_lt
            del temp_lt
            count += 1
    # Decode All the beam indexes to till <END> token only and convert into sentence
    for i in range(beam_width):
        try:
            end_index = beam_indexes[i].index(tokenizer.special_tokens["<END>"])
        except ValueError:
            end_index = len(beam_indexes[i])
            
        decoded_sentences.append(tokenizer.decode(beam_indexes[i][:end_index]))
        
    return decoded_sentences

In [8]:
def get_best_sentence(input_sentences, sentiment=1):
    """
    This function selects the sentence from the Beam of the sentences,
    based on the classification probability score.
    
    input_sentences : list of strings : Sentences generated by the Beam search decoding
    sentiment: int : Expected sentiment (in general class for the classification)
    """
    # BERT pre-processing
    ids = []
    segment_ids = []
    input_masks = []
    pred_lt = []
    for sen in input_sentences:
        text_tokens = tokenizer_cls.tokenize(sen)
        tokens = ["[CLS]"] + text_tokens + ["[SEP]"]
        temp_ids = tokenizer_cls.convert_tokens_to_ids(tokens)
        input_mask = [1] * len(temp_ids)
        segment_id = [0] * len(temp_ids)
        padding = [0] * (max_seq_len - len(temp_ids))

        temp_ids += padding
        input_mask += padding
        segment_id += padding
        
        ids.append(temp_ids)
        input_masks.append(input_mask)
        segment_ids.append(segment_id)
    
    ids = torch.tensor(ids).to(device)
    segment_ids = torch.tensor(segment_ids).to(device)
    input_masks = torch.tensor(input_masks).to(device)
    # prediction
    with torch.no_grad():
        preds = sm(model_cls(ids, segment_ids, input_masks))
        
    preds = preds.tolist()
    inx, inx_val = None, 0
    for i in range(len(input_sentences)):
        temp = preds[i][sentiment]
        if temp > inx_val:
            inx = i
            inx_val = temp
    return input_sentences[inx]

## Example

In [12]:
op=preditction_with_beam_search("<POS> <CON_START> it is not terrible , but it is very good . <START>",4)
op

['it is not terrible , but it is very good .',
 'it is not terrible , but it is very bad .',
 'it is not terrible , but it is not very good .',
 'it is not terrible , but it is very good .']

In [12]:
get_best_sentence(op)

"it is n ' t terrible , but it is very good ."

In [11]:
op=preditction_with_beam_search("<POS> <CON_START> The movie Terminator was very bad . <START>",4)
print(op)
get_best_sentence(op, sentiment=1)

["the movie terminator was n ' t very bad either .", "the movie terminator was n ' t very bad .", "the movie terminator was n ' t very bad .", 'the movie terminator was very bad .']


"the movie terminator was n ' t very bad ."

In [9]:
def preprocess_test_line(line, change_to_sentiment=1):
    #Diego made this
    #
    # So sentiment_test_0.txt looks like
    #   <NEG> <CON_START> ever since joes has changed hands it ' s gotten and . <START> ever since joes has changed hands it 's just gotten worse and worse . <END>
    
    # We'd like to generate a positive example ( based on change_to_sentiment) so we need to change it to:
    #   <POS> <CON_START> ever since joes has changed hands it ' s gotten and . <START>
    if change_to_sentiment == 1:
        #change from NEG to POS
        line = line.replace("<NEG>","<POS>")
    else:
        #change from POS to NEG
        line = line.replace("<POS>","<NEG>")
    
    # now remove stuff after <START> and return processed line
    return line.split("<START>")[0] + "<START>"

test_line = "<NEG> <CON_START> ever since joes has changed hands it ' s gotten and . <START> ever since joes has changed hands it 's just gotten worse and worse . <END>"
print(preprocess_test_line(test_line, change_to_sentiment=1))

<POS> <CON_START> ever since joes has changed hands it ' s gotten and . <START>


## Predictions for the test files

## 1. using B-GST for yelp

In [14]:
# FOR YELP
output_dir = './data/yelp/'

# SEE REFERENCE CELLS FOR NOTES

# sentiment_test_0.txt looks like
#  <NEG> <CON_START> ever since joes has changed hands it ' s gotten and . <START> ever since joes has changed hands it 's just gotten worse and worse . <END>
#  <NEG> <CON_START> there is definitely enough in that part of the venue . <START> there is definitely not enough room in that part of the venue . <END>
#  <NEG> <CON_START> so basically tasted . <START> so basically tasted watered down . <END>
#  <NEG> <CON_START> she she ' d be and for a few minutes . <START> she said she 'd be back and disappeared for a few minutes . <END>
#  <NEG> <CON_START> i ca ' t believe how this pharmacy is . <START> i ca n't believe how inconsiderate this pharmacy is . <END>
#  <NEG> <CON_START> just and took it the bill . <START> just left and took it off the bill . <END>
#  <NEG> <CON_START> it is n ' t terrible , but it is n ' either . <START> it is n't terrible , but it is n't very good either . <END>
#  <NEG> <CON_START> definitely that i could use my birthday gift ! <START> definitely disappointed that i could not use my birthday gift ! <END>
#  <NEG> <CON_START> owner , i heard - but i do ' t know the . <START> new owner , i heard - but i do n't know the details . <END>
#  <NEG> <CON_START> but it sucks ! <START> but it probably sucks too ! <END>

# we want to take these negative ones and make them positive so replace <NEG> with <POS>

with open(os.path.join(output_dir,"bgst_test_0_predictions_with_beam_search.txt") ,"w", encoding='utf-8') as out_fp:
    c = 0
    with open(os.path.join(output_dir, "./processed_files_with_bert_with_best_head/sentiment_test_0.txt")) as fp:
        for line in fp:
            pline = preprocess_test_line(line, change_to_sentiment=1) #change from neg to pos to generate pos pred
            out_sen = preditction_with_beam_search(pline.strip(), beam_width=5, vocab_length=max(tokenizer.special_tokens.values()) + 1)
            if c < 10:
                print(c,line)
                print(pline)
                print(get_best_sentence(out_sen, sentiment = 1))
            c += 1
            out_fp.write(get_best_sentence(out_sen, sentiment = 1) + "\n")

0 <NEG> <CON_START> ever since joes has changed hands it ' s gotten and . <START> ever since joes has changed hands it 's just gotten worse and worse . <END>

<POS> <CON_START> ever since joes has changed hands it ' s gotten and . <START>
ever since joes has changed hands it ' s always gotten better and better .
1 <NEG> <CON_START> there is definitely enough in that part of the venue . <START> there is definitely not enough room in that part of the venue . <END>

<POS> <CON_START> there is definitely enough in that part of the venue . <START>
there is definitely something good enough in that part of the venue .
2 <NEG> <CON_START> so basically tasted . <START> so basically tasted watered down . <END>

<POS> <CON_START> so basically tasted . <START>
so basically tasted delicious .
3 <NEG> <CON_START> she she ' d be and for a few minutes . <START> she said she 'd be back and disappeared for a few minutes . <END>

<POS> <CON_START> she she ' d be and for a few minutes . <START>
she said s

In [15]:
# sentiment_test_1.txt looks like

#<POS> <CON_START> it ' s small yet they you at home . <START> it 's small yet they make you feel right at home . <END>
#<POS> <CON_START> i will be going back and this place ! <START> i will be going back and enjoying this great place ! <END>
#<POS> <CON_START> the drinks were affordable and a . <START> the drinks were affordable and a good pour . <END>
#<POS> <CON_START> my husband got a ruben , he it . <START> my husband got a ruben sandwich , he loved it . <END>
#<POS> <CON_START> i up for their email and a coupon . <START> i signed up for their email and got a coupon . <END>
#<POS> <CON_START> i ' d giving them a try . <START> i 'd definitely recommend giving them a try . <END>
#<POS> <CON_START> i highly e & m . <START> i highly recommend e & m painting . <END>
#<POS> <CON_START> a experience and we will go again . <START> otherwise a great experience and we will go again . <END>
#<POS> <CON_START> drinks , and company . <START> good drinks , and good company . <END>
#<POS> <CON_START> oh i got my band geek now ! <START> oh i got my band geek back on now ! <END>

with open(os.path.join(output_dir,"bgst_test_1_predictions_with_beam_search.txt") ,"w", encoding='utf-8') as out_fp:
    c = 0
    with open(os.path.join(output_dir, "./processed_files_with_bert_with_best_head/sentiment_test_1.txt")) as fp:
        for line in fp:
            pline = preprocess_test_line(line, change_to_sentiment=0) #change from pos to neg to generate neg pred
            out_sen = preditction_with_beam_search(pline.strip(), beam_width=5, vocab_length=max(tokenizer.special_tokens.values()) + 1)
            if c < 10:
                print(c,line)
                print(pline)
                print(c,get_best_sentence(out_sen, sentiment = 0))
            c += 1
            out_fp.write(get_best_sentence(out_sen, sentiment = 0) + "\n")

0 <POS> <CON_START> it ' s small yet they you at home . <START> it 's small yet they make you feel right at home . <END>

<NEG> <CON_START> it ' s small yet they you at home . <START>
0 it ' s small yet they treat you like crap at home .
1 <POS> <CON_START> i will be going back and this place ! <START> i will be going back and enjoying this great place ! <END>

<NEG> <CON_START> i will be going back and this place ! <START>
1 i will be going back and this place is horrible !
2 <POS> <CON_START> the drinks were affordable and a . <START> the drinks were affordable and a good pour . <END>

<NEG> <CON_START> the drinks were affordable and a . <START>
2 the drinks were affordable and a little overpriced .
3 <POS> <CON_START> my husband got a ruben , he it . <START> my husband got a ruben sandwich , he loved it . <END>

<NEG> <CON_START> my husband got a ruben , he it . <START>
3 my husband got a ruben , he got it wrong .
4 <POS> <CON_START> i up for their email and a coupon . <START> i sig

## B-GST for caption

In [9]:
# FOR Image Caption
output_dir = './data/imagecaption/'

# SEE REFERENCE CELLS FOR NOTES

# sentiment_test_0.txt looks like
"""
<NEG> <CON_START> the group of is resting in front of a . <START> the group of hikers is resting in front of a mountain . <END>
<NEG> <CON_START> a boy carrying goggles . <START> a wet boy carrying goggles runs . <END>
<NEG> <CON_START> little slides into plate where rival crouches in . <START> little league slides into plate where rival player crouches in anticipation . <END>
<NEG> <CON_START> children are on a playground slide . <START> three children are posing on a playground slide . <END>
<NEG> <CON_START> on a mosaic on the ground . <START> artists working on a mosaic on the ground . <END>
<NEG> <CON_START> a white bird flies the backdrop of . <START> a white bird flies agains the backdrop of green foliage . <END>
<NEG> <CON_START> a has an apron on its back . <START> a miniature dachshund has an apron on its back . <END>
<NEG> <CON_START> a flies through the air in front of . <START> a pelican flies through the air in front of trees . <END>
<NEG> <CON_START> by the water ' s edge . <START> two people stand by the water 's edge . <END>
<NEG> <CON_START> brown black dog through snow . <START> brown dog chasing black dog through snow . <END>
"""
# we want to take these negative ones and make them positive so replace <NEG> with <POS>

with open(os.path.join(output_dir,"bgst_test_0_predictions_with_beam_search.txt") ,"w", encoding='utf-8') as out_fp:
    c = 0
    with open(os.path.join(output_dir, "./processed_files_with_bert_with_best_head/sentiment_test_0.txt")) as fp:
        for line in fp:
            pline = preprocess_test_line(line, change_to_sentiment=1) #change from neg to pos to generate pos pred
            out_sen = preditction_with_beam_search(pline.strip(), beam_width=5, vocab_length=max(tokenizer.special_tokens.values()) + 1)
            if c < 10:
                print(c,line)
                print(pline)
                print(get_best_sentence(out_sen, sentiment = 1))
            c += 1
            out_fp.write(get_best_sentence(out_sen, sentiment = 1) + "\n")        

0 <NEG> <CON_START> the group of is resting in front of a . <START> the group of hikers is resting in front of a mountain . <END>

<POS> <CON_START> the group of is resting in front of a . <START>
the group of people is resting peacefully in front of a building .
1 <NEG> <CON_START> a boy carrying goggles . <START> a wet boy carrying goggles runs . <END>

<POS> <CON_START> a boy carrying goggles . <START>
a boy carrying goggles smiles proudly .
2 <NEG> <CON_START> little slides into plate where rival crouches in . <START> little league slides into plate where rival player crouches in anticipation . <END>

<POS> <CON_START> little slides into plate where rival crouches in . <START>
little boy slides into plate where rival crouches in fear .
3 <NEG> <CON_START> children are on a playground slide . <START> three children are posing on a playground slide . <END>

<POS> <CON_START> children are on a playground slide . <START>
two children are playing on a playground slide .
4 <NEG> <CON_STA

In [10]:
# sentiment_test_1.txt looks like

"""
<POS> <CON_START> the group of is resting in front of a . <START> the group of hikers is resting in front of a mountain . <END>
<POS> <CON_START> a boy carrying goggles . <START> a wet boy carrying goggles runs . <END>
<POS> <CON_START> little slides into plate where rival crouches in . <START> little league slides into plate where rival player crouches in anticipation . <END>
<POS> <CON_START> children are on a playground slide . <START> three children are posing on a playground slide . <END>
<POS> <CON_START> on a mosaic on the ground . <START> artists working on a mosaic on the ground . <END>
<POS> <CON_START> a white bird flies the backdrop of . <START> a white bird flies agains the backdrop of green foliage . <END>
<POS> <CON_START> a has an apron on its back . <START> a miniature dachshund has an apron on its back . <END>
<POS> <CON_START> a flies through the air in front of . <START> a pelican flies through the air in front of trees . <END>
<POS> <CON_START> by the water ' s edge . <START> two people stand by the water 's edge . <END>
<POS> <CON_START> brown black dog through snow . <START> brown dog chasing black dog through snow . <END>
"""

with open(os.path.join(output_dir,"bgst_test_1_predictions_with_beam_search.txt") ,"w", encoding='utf-8') as out_fp:
    c = 0
    with open(os.path.join(output_dir, "./processed_files_with_bert_with_best_head/sentiment_test_1.txt")) as fp:
        for line in fp:
            pline = preprocess_test_line(line, change_to_sentiment=0) #change from pos to neg to generate neg pred
            out_sen = preditction_with_beam_search(pline.strip(), beam_width=5, vocab_length=max(tokenizer.special_tokens.values()) + 1)
            if c < 10:
                print(c,line)
                print(pline)
                print(c,get_best_sentence(out_sen, sentiment = 0))
            c += 1
            out_fp.write(get_best_sentence(out_sen, sentiment = 0) + "\n")

0 <POS> <CON_START> the group of is resting in front of a . <START> the group of hikers is resting in front of a mountain . <END>

<NEG> <CON_START> the group of is resting in front of a . <START>
0 the group of people is resting in front of a waterfall .
1 <POS> <CON_START> a boy carrying goggles . <START> a wet boy carrying goggles runs . <END>

<NEG> <CON_START> a boy carrying goggles . <START>
1 a boy carrying goggles .
2 <POS> <CON_START> little slides into plate where rival crouches in . <START> little league slides into plate where rival player crouches in anticipation . <END>

<NEG> <CON_START> little slides into plate where rival crouches in . <START>
2 little slides into plate where rival crouches in search of bones .
3 <POS> <CON_START> children are on a playground slide . <START> three children are posing on a playground slide . <END>

<NEG> <CON_START> children are on a playground slide . <START>
3 children are playing on a playground slide .
4 <POS> <CON_START> on a mosai

## B-GST for AMAZON

In [11]:
# FOR Image Caption
output_dir = './data/amazon/'

# SEE REFERENCE CELLS FOR NOTES

# data/amazon/processed_files_with_bert_with_best_head/sentiment_test_0.txt looks like
"""
<NEG> <CON_START> your had a kickstand on the plate it won t down . <START> if your bike had a kickstand on the plate it won t lock down .  <END>
<NEG> <CON_START> they are and memory than the older ones . <START> they are thinner and retain memory than the older ones .  <END>
<NEG> <CON_START> it makes a sound devices are plugged in . <START> it makes a buzzing sound when devices are plugged in .  <END>
<NEG> <CON_START> i could barely get it they so nasty . <START> i could barely get through it they taste so nasty .  <END>
<NEG> <CON_START> either than the to do their own . <START> either than or the developers decided to do their own .  <END>
<NEG> <CON_START> the pair in mexico is much . <START> the pair made in mexico is much better .  <END>
<NEG> <CON_START> i m mine to make for something . <START> i m selling mine to make room for something else .  <END>
<NEG> <CON_START> it looks like i ve glitter my face . <START> it looks like i ve splashed glitter all over my face .  <END>
<NEG> <CON_START> went the second one using the same , no . <START> went to the second one using the same method ,  no go .  <END>
<NEG> <CON_START> it was as good as our much cheaper . <START> it was not as good as our much cheaper model .  <END>
"""
# we want to take these negative ones and make them positive so replace <NEG> with <POS>

with open(os.path.join(output_dir,"bgst_test_0_predictions_with_beam_search.txt") ,"w", encoding='utf-8') as out_fp:
    c = 0
    with open(os.path.join(output_dir, "./processed_files_with_bert_with_best_head/sentiment_test_0.txt")) as fp:
        for line in fp:
            pline = preprocess_test_line(line, change_to_sentiment=1) #change from neg to pos to generate pos pred
            out_sen = preditction_with_beam_search(pline.strip(), beam_width=5, vocab_length=max(tokenizer.special_tokens.values()) + 1)
            if c < 10:
                print(c,line)
                print(pline)
                print(get_best_sentence(out_sen, sentiment = 1))
            c += 1
            out_fp.write(get_best_sentence(out_sen, sentiment = 1) + "\n")        

0 <NEG> <CON_START> your had a kickstand on the plate it won t down . <START> if your bike had a kickstand on the plate it won t lock down .  <END>

<POS> <CON_START> your had a kickstand on the plate it won t down . <START>
if your had a kickstand on the plate it won t slide down .
1 <NEG> <CON_START> they are and memory than the older ones . <START> they are thinner and retain memory than the older ones .  <END>

<POS> <CON_START> they are and memory than the older ones . <START>
they are better and better memory than the older ones .
2 <NEG> <CON_START> it makes a sound devices are plugged in . <START> it makes a buzzing sound when devices are plugged in .  <END>

<POS> <CON_START> it makes a sound devices are plugged in . <START>
it makes a clear sound when devices are plugged in .
3 <NEG> <CON_START> i could barely get it they so nasty . <START> i could barely get through it they taste so nasty .  <END>

<POS> <CON_START> i could barely get it they so nasty . <START>
i could barel

In [12]:
# sentiment_test_1.txt looks like

"""
<POS> <CON_START> i ve had this thermometer for num _ num . <START> i ve had this thermometer for about num_num years .  <END>
<POS> <CON_START> will update this review should it meet a similar to my . <START> will update this review should it meet a similar fate to my previous wheel .  <END>
<POS> <CON_START> this product what it is suppose to . <START> this product does what it is suppose to do .  <END>
<POS> <CON_START> i this filter a num _ extend coffeemaker . <START> i purchased this filter for a braun num_extend coffeemaker .  <END>
<POS> <CON_START> very good price this to use than my cusinart . <START> very good price but this one harder to use than my cusinart .  <END>
<POS> <CON_START> so , me this is pretty darn . <START> so ,  to me this is pretty darn  instant  .  <END>
<POS> <CON_START> my pampered chef garlic press with this . <START> replaced my pampered chef garlic press with this one .  <END>
<POS> <CON_START> i this a little office supply shelf . <START> i use this as a little office supply shelf .  <END>
<POS> <CON_START> and similar quality to apple products . <START> durable and seems similar quality to apple products .  <END>
<POS> <CON_START> i have oxo products , and i ve pleased . <START> i have many oxo products ,  and i ve always been pleased .  <END>
"""

with open(os.path.join(output_dir,"bgst_test_1_predictions_with_beam_search.txt") ,"w", encoding='utf-8') as out_fp:
    c = 0
    with open(os.path.join(output_dir, "./processed_files_with_bert_with_best_head/sentiment_test_1.txt")) as fp:
        for line in fp:
            pline = preprocess_test_line(line, change_to_sentiment=0) #change from pos to neg to generate neg pred
            out_sen = preditction_with_beam_search(pline.strip(), beam_width=5, vocab_length=max(tokenizer.special_tokens.values()) + 1)
            if c < 10:
                print(c,line)
                print(pline)
                print(c,get_best_sentence(out_sen, sentiment = 0))
            c += 1
            out_fp.write(get_best_sentence(out_sen, sentiment = 0) + "\n")

0 <POS> <CON_START> i ve had this thermometer for num _ num . <START> i ve had this thermometer for about num_num years .  <END>

<NEG> <CON_START> i ve had this thermometer for num _ num . <START>
0 i ve had this thermometer for about num _ num years now .
1 <POS> <CON_START> will update this review should it meet a similar to my . <START> will update this review should it meet a similar fate to my previous wheel .  <END>

<NEG> <CON_START> will update this review should it meet a similar to my . <START>
1 will update this review should it meet a similar fate to my previous one .
2 <POS> <CON_START> this product what it is suppose to . <START> this product does what it is suppose to do .  <END>

<NEG> <CON_START> this product what it is suppose to . <START>
2 this product does not do what it is suppose to .
3 <POS> <CON_START> i this filter a num _ extend coffeemaker . <START> i purchased this filter for a braun num_extend coffeemaker .  <END>

<NEG> <CON_START> i this filter a num _ 

## Predictions for the reference files

In [13]:
# FOR SAVING PURPOSES SET OUTPUT DIR( however it must contain reference_0.txt and reference_1.txt )
#output_dir = "/home/ubuntu/bhargav/data/yelp/processed_files_with_bert_with_best_head/"
#output_dir = "/Users/diegoolano/Documents/UT/semester6/to_read_RL_proj_etc/evaluating/TRDG/data/yelp"
output_dir = './data/yelp/'
os.listdir(output_dir)

#all predictions made by 2 Epoch Trained BERT Classifier

['bert_classifier_training',
 'change_filenames.sh',
 'sentiment_dev_1.txt',
 'sentiment_train_1.txt',
 'reference_0.txt',
 'sentiment_test_0.txt',
 'sentiment_dev_0.txt',
 'bert_classifier',
 'bert_classifier_3epochs',
 'bert_classifier_2epochs',
 'sentiment_test_1.txt',
 'bert_classifier_1000epochs',
 'reference_0_predictions_with_beam_search.txt',
 'bert_classifier_100epochs',
 'reference_1.txt',
 'sentiment_train_0.txt',
 'processed_files_with_bert_with_best_head']

In [14]:
#this takes negative sentences and generates positive ones

# first three examples from reference_0.txt ( first column is negative  ... second is positive)
# *****************************************************
# ever since joes has changed hands it 's just gotten worse and worse .	Ever since joes has changed hands it's gotten better and better.
# there is definitely not enough room in that part of the venue .	There is so much room in that part of the venue
# so basically tasted watered down .	It didn't taste watered down at all. 

# QUESTION: does there code expect both? 
# NOPE! these references need to be formated similar to what we see in the example above ! 
# <POS> <CON_START> ... <START>"
 
# A. during training of B-GST models
# they take a training example as follows: 
#   Training input: <POS> <CON_START> The food was and the service was <START> The food was great and the service was excellent . <END>

# the model sees the target style "<POS>" (ie, what "target style" we'd like to generate)
# then the CONtent String ( ie, sentence with no attributes ) 
# and tries to generate the Full String given(post <START>)

#Prepare training and inference data:
# 1. Run BERT_DATA_PREPARATION.ipynb for preparing training and inference data for Delete and Generate . 
# Use the best layer, Head combination from the previous step in run_attn_examples() function.

# DONE... NOW WE NEED TO USE reference files from  data/yelp/processed_files_with_bert_with_best_head
# *****************************************************
#<POS> <CON_START> ever since joes has changed hands it ' s just gotten and . ever since joes has changed hands it ' s gotten and better . <START>
#<POS> <CON_START> there is definitely enough in that part of the venue . there is so much in that part of the venue <START>
#<POS> <CON_START> so basically tasted . it didn ' t taste down at all . <START>


with open(os.path.join(output_dir,"reference_0_predictions_with_beam_search.txt") ,"w", encoding='utf-8') as out_fp:
    c = 0
    with open(os.path.join(output_dir, "./processed_files_with_bert_with_best_head/reference_0.txt")) as fp:
        for line in fp:
            out_sen = preditction_with_beam_search(line.strip(), beam_width=5, vocab_length=max(tokenizer.special_tokens.values()) + 1)
            print(c,get_best_sentence(out_sen, sentiment = 1))
            c += 1
            out_fp.write(get_best_sentence(out_sen, sentiment = 1) + "\n")

0 ever since joes has changed hands it ' s just gotten better and better .
1 there is pretty much everything in that part of the venue .
2 so basically tasted good tasted good .
3 she said she ' ll definitely be back and for a few minutes .
4 i ca ' t believe how amazing this pharmacy is .
5 just went and took it the bill and put it on the bill .
6 it is n ' t terrible , but it is n ' t very expensive either .
7 definitely recommend that i could use my birthday gift right now !
8 owner , i heard - but i do n ' t know the details .
9 but it probably does n ' t suck too bad too !
10 we sat down and we got some really good and friendly service .
11 very good charge did include miso soup and a small salad .
12 they asked how everything came out .
13 we could eat at the table if we were n ' t ordering dinner .
14 the cash register area was very nice and one was the store front .
15 chips are really good , but their salsa is really good .
16 the food was even better .
17 mainly by teenagers 

In [16]:
with open(os.path.join(output_dir,"reference_1_predictions_with_beam_search.txt") ,"w", encoding='utf-8') as out_fp:
    c = 0
    with open(os.path.join(output_dir, "./processed_files_with_bert_with_best_head/reference_1.txt")) as fp:
        for line in fp:
            out_sen = preditction_with_beam_search(line.strip(), beam_width=5, vocab_length=max(tokenizer.special_tokens.values()) + 1)
            print(c,get_best_sentence(out_sen, sentiment = 0))
            c += 1
            out_fp.write(get_best_sentence(out_sen, sentiment = 0) + "\n")

0 it ' s small yet they make you feel like a stranger at home .
1 i wo n ' t be going back and will avoid this place forever !
2 the drinks were affordable and a little over priced .
3 my husband got a ruben sandwich , he did n ' t like it .
4 i signed up for their email and left a coupon .
5 i ' d probably rather give them a try elsewhere .
6 i highly recommend e & m using e & m .
7 otherwise a disaster and we will never go again .
8 no drinks , no company .
9 oh i do n ' t get my band geek on now !
10 i was nervous and she made me feel so uncomfortable and uncomfortable .
11 the tuna and tuna were both very bland .
12 everything is n ' t fresh and so gross !
13 the biscuits and gravy was n ' t good .
14 i would n ' t even wait next year !
15 they were so rude , rude , and reasonably priced .
16 i do n ' t like pei wei .
17 unfortunately , authentic bagels , tasty cream cheese , and the cream cheese were n ' t very tasty .
18 you ' ll understand why once you get there .
19 everyone th

# 2. Delete, Retrieve and Generate  ( G-GST )

In [13]:
special_tokens = ['<ATTR_WORDS>','<CON_START>','<START>','<END>'] # Set the special tokens
tokenizer = OpenAIGPTTokenizer.from_pretrained('openai-gpt', special_tokens=special_tokens)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = OpenAIGPTLMHeadModel.from_pretrained('openai-gpt', num_special_tokens=len(special_tokens))

ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.


In [14]:
## 1. Set Model Path to Downloaded Weights
model_path = "./emnlp_weights/G-GST/pytorch_model_zero_grad_1.bin"  #YELP
model_path = "./data/imagecaption/ggst_classifier/pytorch_model_zero_grad_1.bin"  #IMAGE CAPTION
model_path = "./data/amazon/ggst_classifier/pytorch_model_zero_grad_1.bin"  #Amazon

path = os.path.join(os.getcwd(), model_path) 
model_state_dict = torch.load(path, map_location=device)
model.load_state_dict(model_state_dict)
model.to(device)
model.eval()

OpenAIGPTLMHeadModel(
  (transformer): OpenAIGPTModel(
    (tokens_embed): Embedding(40482, 768)
    (positions_embed): Embedding(512, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_1): BertLayerNorm()
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): BertLayerNorm()
      )
      (1): Block(
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_1): BertLayerNorm()
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout

In [15]:
#bert_classifier_dir = "./data/yelp/bert_classifier_2epochs/"   #gives 98% acc  yelp
#bert_classifier_dir = "./data/imagecaption/bert_classifier_10epochs/"
bert_classifier_dir = "./data/amazon/bert_classifier_3epochs/"

model_cls = BertForSequenceClassification.from_pretrained(bert_classifier_dir, num_labels=2)
tokenizer_cls = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
model_cls.to(device)
model_cls.eval()

max_seq_len=70   ### IS THIS THE ISSUE ?   I thought it trained at a different size?  this is just for padding though

sm = torch.nn.Softmax(dim=-1)
print(model.config.n_positions)
# NOW LOAD TWO FUNCTIONS ABOVE preditction_with_beam_search and get_best_sentence

512


In [16]:
def ggst1_pred_with_beam_search(ref_text, beam_width=3, vocab_length=40483, debug=False):
    """
    This function decodes sentences using Beam Seach. 
    It will output #sentences = beam_width. This function works on a single example.
    
    ref_text : string : Input sentence
    beam_width : int : Width of the output beam
    vocab_length : int : Size of the Vocab after adding the special tokens
    """
    
    done = [False for i in range(beam_width)] # To track which beams are already decoded
    stop_decode = False
    decoded_sentences=[] # List of decoded sentences at any given time
    
    sm = torch.nn.Softmax(dim=-1) # To calculate Softmax over the final layer Logits   #NEED TO DO THIS OUTSIDE ?
    
    tokens = tokenizer.tokenize(ref_text) # Tokenize the input text
    
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokens) # Convert tokens to ids
    index_tokens = [indexed_tokens for i in range(beam_width)] # Replication of Input ids for all the beams

    torch_tensor = torch.tensor(index_tokens).to(device)
    beam_indexes = [[] for i in range(beam_width)] # indexes of the current decoded beams
    best_scoes = [0 for i in range(beam_width)] # A list of lists to store Probability values of each decoded token of best beams
    count = 0
    
    if debug:
        print(ref_text,len(tokens),tokens,len(indexed_tokens),indexed_tokens)
        print(sm)
    
    while count < model.config.n_positions and not stop_decode:
        
        if count == 0: # For the first step when only one sentence is availabe
            with torch.no_grad():
                # Calculate output probability distribution over the Vocab,
                preds = sm(model(torch_tensor)) #  shape = [beam_bidth, len(input_sen)+1,Vocab_length]
            top_v, top_i = preds[:,-1,:].topk(beam_width) # Fatch top indexes and it's values
            [beam_indexes[i].append(top_i[0][i].tolist()) for i in range(beam_width)] # Update the Beam indexes
            # Update the best_scores, for first time just add the topk values directly
            for i in range(beam_width):
                best_scoes[i] = top_v[0][i].item()
            count += 1
        else: # After first step
            
            # Prepare the current_state by concating original input and decoded beam indexes
            current_state = torch.cat((torch_tensor, torch.tensor(beam_indexes).to(device)), dim=1)
            
            # Prediction on the current state
            with torch.no_grad():
                preds = sm(model(current_state))
                
            # Multiply new probability predictions with corresponding best scores
            # Total socres = beam_width * Vocab_Size
            flatten_score = (preds[:,-1,:]*torch.tensor(best_scoes).to(device).unsqueeze(1)).view(-1)
            
            # Fetch the top scores and indexes 
            vals, inx = flatten_score.topk(beam_width)
            
            # best_score_inx saves the index of best beams after multiplying the probability of new prediction
            best_scoes_inx = (inx / vocab_length).tolist()
            best_scoes = vals.tolist()
            
            # Unflatten the index 
            correct_inx = (inx % vocab_length).tolist()
            
            # Check if done for all the Beams
            for i in range(beam_width):
                if correct_inx[i] == tokenizer.special_tokens["<END>"]:
                    done[i] = True
                    
            # Update the best score for each the current Beams
            for i in range(beam_width):
                if not done[i]:
                    best_scoes[i] = vals.tolist()[i]
                    
            # Check is All the Beams are Done
            if (sum(done) == beam_width):
                stop_decode = True
                
            # Prepapre the new beams
            temp_lt=[0 for i in range(beam_width)]
            for i,x in enumerate(best_scoes_inx):
                temp_lt[i] = beam_indexes[x] + [correct_inx[i]]
                
            # Update the Beam indexes
            beam_indexes = temp_lt
            del temp_lt
            count += 1
            
    # Decode All the beam indexes to till <END> token only and convert into sentence
    for i in range(beam_width):
        try:
            end_index = beam_indexes[i].index(tokenizer.special_tokens["<END>"])
        except ValueError:
            end_index = len(beam_indexes[i])
            
        decoded_sentences.append(tokenizer.decode(beam_indexes[i][:end_index]))
        
    return decoded_sentences


def get_best_sentence(input_sentences, sentiment=1):
    """
    This function selects the sentence from the Beam of the sentences,
    based on the classification probability score.
    
    input_sentences : list of strings : Sentences generated by the Beam search decoding
    sentiment: int : Expected sentiment (in general class for the classification)
    """
    # BERT pre-processing
    ids = []
    segment_ids = []
    input_masks = []
    pred_lt = []
    for sen in input_sentences:
        text_tokens = tokenizer_cls.tokenize(sen)
        tokens = ["[CLS]"] + text_tokens + ["[SEP]"]
        temp_ids = tokenizer_cls.convert_tokens_to_ids(tokens)
        input_mask = [1] * len(temp_ids)
        segment_id = [0] * len(temp_ids)
        padding = [0] * (max_seq_len - len(temp_ids))

        temp_ids += padding
        input_mask += padding
        segment_id += padding
        
        ids.append(temp_ids)
        input_masks.append(input_mask)
        segment_ids.append(segment_id)
    
    ids = torch.tensor(ids).to(device)
    segment_ids = torch.tensor(segment_ids).to(device)
    input_masks = torch.tensor(input_masks).to(device)
    
    # prediction
    with torch.no_grad():
        preds = sm(model_cls(ids, segment_ids, input_masks))   #it was failing here due to lack of SM
        
    preds = preds.tolist()
    inx, inx_val = None, 0
    for i in range(len(input_sentences)):
        temp = preds[i][sentiment]
        if temp > inx_val:
            inx = i
            inx_val = temp
    return input_sentences[inx]

In [17]:
#1. Run Delete_Retrieve_Generate_Data_Preparation.ipynb to generate training data for Delete, Retrieve and Generate . 
#  It generates train, dev and test files. Use the files generated by process_file_v1() function 
#  as it shuffles the attributes and randomly samples only 70% of the attributes to train the generator model 
#  to generate smooth sentences instead of teaching it to just fill the blanks.

#2. Run tfidf_retrieve.ipynb to generate inference data by retrieving attributes of closest match from target style training corpus.




# Output dir have reference files generated using TFIDF for retrieve attributes from opposite corpus
#output_dir = "/home/ubuntu/bhargav/data/yelp/processed_files_with_bert_with_best_head/delete_retrieve_edit_model/tfidf/"

#output_dir = "./data/yelp/processed_files_with_bert_with_best_head/delete_retrieve_edit_model/tfidf/"
#output_dir = "./data/imagecaption/processed_files_with_bert_with_best_head/delete_retrieve_edit_model/tfidf/"
output_dir = "./data/amazon/processed_files_with_bert_with_best_head/delete_retrieve_edit_model/tfidf/"

os.listdir(output_dir)

['test_0.txt', 'test_1.txt']

## Generate Texts for Test data

# 2. G-GST yelp

In [24]:
#g_gst_test0_f = "processed_files_with_bert_with_best_head/delete_retrieve_edit_model/sentiment_test_0.txt"

#looks like
# <ATTR_WORDS> worse 's <CON_START> ever since joes has changed hands it ' s gotten and . <START> ever since joes has changed hands it 's just gotten worse and worse . <END>
# <ATTR_WORDS> not room <CON_START> there is definitely enough in that part of the venue . <START> there is definitely not enough room in that part of the venue . <END>

# SO I NEED TO RUN tfidf_retrieve  on the test data !

# tfidf/test_0  contains the negative test sentences which have been given positive attributes from train
# tfidf/test_1  contains the positive test sentences which have been given negative attributes from train

#bgst_test_0_predictions_with_beam_search.txt  does what test_0 does

od = "./data/yelp/processed_files_with_bert_with_best_head/delete_retrieve_edit_model/tfidf/"
g_gst_test0_f = "test_0.txt"
g_gst_test1_f = "test_1.txt"

with open(os.path.join(od,"ggst_test_0_predictions_with_full_sentence_match_beam_search_bm5.txt") ,"w", encoding='utf-8') as out_fp:
    c = 0
    with open(os.path.join(od, g_gst_test0_f)) as fp:
        for line in fp:
            out_sen = preditction_with_beam_search(line.strip(), beam_width=5, vocab_length=max(tokenizer.special_tokens.values()) + 1)
            if c < 10:
                print(c,line)
                print(out_sen)
                print(get_best_sentence(out_sen, sentiment = 1))
            c += 1
            out_fp.write(get_best_sentence(out_sen, sentiment = 1) + "\n")

0 <ATTR_WORDS> however tidy decent <CON_START> ever since joes has changed hands it ' s gotten and . <START>

["however ever since joes has changed hands it ' s gotten decent and friendly .", "however ever since joes has changed hands it ' s gotten decent and clean .", "however ever since joes has changed hands it ' s gotten decent and friendly .", "however ever since joes has changed hands it ' s gotten tidy and clean .", "however ever since joes has changed hands it ' s gotten tidy and friendly ."]
however ever since joes has changed hands it ' s gotten decent and friendly .
1 <ATTR_WORDS> great place <CON_START> there is definitely enough in that part of the venue . <START>

['there is definitely great enough place in that part of the venue .', 'there is definitely great enough place in that part of the venue .', 'there is definitely great enough place in that part of the venue .', 'there is definitely great enough place in that part of the venue .', 'there is definitely great enoug

In [25]:
with open(os.path.join(od,"ggst_test_1_predictions_with_full_sentence_match_beam_search_bm5.txt") ,"w", encoding='utf-8') as out_fp:
    c = 0
    with open(os.path.join(od, g_gst_test1_f)) as fp:
        for line in fp:
            out_sen = preditction_with_beam_search(line.strip(), beam_width=5, vocab_length=max(tokenizer.special_tokens.values()) + 1)
            if c < 10:
                print(c,line)
                print(out_sen)
                print(get_best_sentence(out_sen, sentiment = 0))
            c += 1
            out_fp.write(get_best_sentence(out_sen, sentiment = 0) + "\n")

0 <ATTR_WORDS> crust cardboard <CON_START> it ' s small yet they you at home . <START>

["it ' s small yet they slice you at home .", "it ' s small but they crust you at home .", "it ' s small crust yet they make you eat at home .", "it ' s small crust but they make you eat at home .", "it ' s small yet they make you eat at home ."]
it ' s small but they crust you at home .
1 <ATTR_WORDS> urge tell off <CON_START> i will be going back and this place ! <START>

['i will be going back and tell this place off !', 'i will be going back and tell this place off !', 'i will be going back and tell this place off !', 'i will be going back and tell this place off !', 'i will be going back and tell this place off !']
i will be going back and tell this place off !
2 <ATTR_WORDS> 's 's did not reservation <CON_START> the drinks were affordable and a . <START>

["the drinks were affordable and did n ' t have a reservation .", "the drinks were affordable and did n ' t have a reservation .", "the drin

# GGST image caption

In [9]:
#g_gst_test0_f = "processed_files_with_bert_with_best_head/delete_retrieve_edit_model/sentiment_test_0.txt"
#looks like
"""
<ATTR_WORDS> hikers mountain <CON_START> the group of is resting in front of a . <START> the group of hikers is resting in fro
nt of a mountain . <END>
<ATTR_WORDS> wet runs <CON_START> a boy carrying goggles . <START> a wet boy carrying goggles runs . <END>
<ATTR_WORDS> anticipation player <CON_START> little slides into plate where rival crouches in . <START> little league slides i
nto plate where rival player crouches in anticipation . <END>
"""

# tfidf/test_0  contains the negative test sentences which have been given positive attributes from train
# tfidf/test_1  contains the positive test sentences which have been given negative attributes from train

#bgst_test_0_predictions_with_beam_search.txt  does what test_0 does

od = "./data/imagecaption/processed_files_with_bert_with_best_head/delete_retrieve_edit_model/tfidf/"
g_gst_test0_f = "test_0.txt"
g_gst_test1_f = "test_1.txt"

with open(os.path.join(od,"ggst_test_0_predictions_with_full_sentence_match_beam_search_bm5.txt") ,"w", encoding='utf-8') as out_fp:
    c = 0
    with open(os.path.join(od, g_gst_test0_f)) as fp:
        for line in fp:
            out_sen = preditction_with_beam_search(line.strip(), beam_width=5, vocab_length=max(tokenizer.special_tokens.values()) + 1)
            if c < 10:
                print(c,line)
                print(out_sen)
                print(get_best_sentence(out_sen, sentiment = 1))
            c += 1
            out_fp.write(get_best_sentence(out_sen, sentiment = 1) + "\n")

0 <ATTR_WORDS> true love , <CON_START> the group of is resting in front of a . <START>

['the group of people are resting in front of a true love , true love .', 'the group of people is resting in front of a true love , true love .', 'the group of people are resting in front of a true love , love .', 'the group of people is resting in front of a true love , true love .', 'the group of people are resting in front of a true love , true love .']
the group of people is resting in front of a true love , true love .
1 <ATTR_WORDS> everyone full joy <CON_START> a boy carrying goggles . <START>

['a boy carrying goggles full of joy .', 'a boy carrying goggles full of joy .', 'a boy carrying goggles full of joy .', 'a boy carrying goggles full of joy .', 'a boy carrying goggles full of joy .']
a boy carrying goggles full of joy .
2 <ATTR_WORDS> eats favorite meal <CON_START> little slides into plate where rival crouches in . <START>

['little slides into plate where rival crouches in search of 

In [10]:
with open(os.path.join(od,"ggst_test_1_predictions_with_full_sentence_match_beam_search_bm5.txt") ,"w", encoding='utf-8') as out_fp:
    c = 0
    with open(os.path.join(od, g_gst_test1_f)) as fp:
        for line in fp:
            out_sen = preditction_with_beam_search(line.strip(), beam_width=5, vocab_length=max(tokenizer.special_tokens.values()) + 1)
            if c < 10:
                print(c,line)
                print(out_sen)
                print(get_best_sentence(out_sen, sentiment = 0))
            c += 1
            out_fp.write(get_best_sentence(out_sen, sentiment = 0) + "\n")

0 <ATTR_WORDS> get from zombies <CON_START> the group of is resting in front of a . <START>

['the group of zombies are resting in front of a zombie to get from .', 'the group of people are resting in front of a zombie to get from .', 'the group of zombies is resting in front of a zombie to get from .', 'the group of people is resting in front of a zombie to get from .', 'the group of people are resting in front of a zombie to get from zombies .']
the group of zombies are resting in front of a zombie to get from .
1 <ATTR_WORDS> searching for mermaids <CON_START> a boy carrying goggles . <START>

['a boy carrying goggles searching for mermaids .', 'a boy carrying goggles searching for mermaids .', 'a boy carrying goggles searching for mermaids .', 'a boy carrying goggles searches for mermaids .', 'a boy carrying goggles searching for mermaids .']
a boy carrying goggles searches for mermaids .
2 <ATTR_WORDS> lake , id. <CON_START> little slides into plate where rival crouches in . <STAR

## GGST amazon

In [18]:
#g_gst_test0_f = "processed_files_with_bert_with_best_head/delete_retrieve_edit_model/sentiment_test_0.txt"
#looks like
"""
<ATTR_WORDS> almost overlooked slides <CON_START> your had a kickstand on the plate it won t down . <START>
<ATTR_WORDS> quality of product <CON_START> they are and memory than the older ones . <START>
<ATTR_WORDS> when first popped num_num <CON_START> it makes a sound devices are plugged in . <START>
<ATTR_WORDS> time eyes good <CON_START> i could barely get it they so nasty . <START>
"""

# tfidf/test_0  contains the negative test sentences which have been given positive attributes from train
# tfidf/test_1  contains the positive test sentences which have been given negative attributes from train

#bgst_test_0_predictions_with_beam_search.txt  does what test_0 does

od = "./data/amazon/processed_files_with_bert_with_best_head/delete_retrieve_edit_model/tfidf/"
g_gst_test0_f = "test_0.txt"
g_gst_test1_f = "test_1.txt"

with open(os.path.join(od,"ggst_test_0_predictions_with_full_sentence_match_beam_search_bm5.txt") ,"w", encoding='utf-8') as out_fp:
    c = 0
    with open(os.path.join(od, g_gst_test0_f)) as fp:
        for line in fp:
            out_sen = preditction_with_beam_search(line.strip(), beam_width=5, vocab_length=max(tokenizer.special_tokens.values()) + 1)
            if c < 10:
                print(c,line)
                print(out_sen)
                print(get_best_sentence(out_sen, sentiment = 1))
            c += 1
            out_fp.write(get_best_sentence(out_sen, sentiment = 1) + "\n")

0 <ATTR_WORDS> almost overlooked slides <CON_START> your had a kickstand on the plate it won t down . <START>

['if your had a kickstand on the plate it almost won t slide down .', 'your daughter had a kickstand on the plate it almost won t slide down .', 'your had a kickstand on the plate it almost won t slide down .', 'your bike had a kickstand on the plate it almost won t slide down .', 'your dog had a kickstand on the plate it almost won t slide down .']
your had a kickstand on the plate it almost won t slide down .
1 <ATTR_WORDS> quality of product <CON_START> they are and memory than the older ones . <START>

['they are of better quality and memory than the older ones .', 'they are better quality and memory than the older ones .', 'they are quality and more memory than the older ones .', 'they are of quality and memory than the older ones .', 'they are quality and memory than the older ones .']
they are of quality and memory than the older ones .
2 <ATTR_WORDS> when first popped 

In [19]:
with open(os.path.join(od,"ggst_test_1_predictions_with_full_sentence_match_beam_search_bm5.txt") ,"w", encoding='utf-8') as out_fp:
    c = 0
    with open(os.path.join(od, g_gst_test1_f)) as fp:
        for line in fp:
            out_sen = preditction_with_beam_search(line.strip(), beam_width=5, vocab_length=max(tokenizer.special_tokens.values()) + 1)
            if c < 10:
                print(c,line)
                print(out_sen)
                print(get_best_sentence(out_sen, sentiment = 0))
            c += 1
            out_fp.write(get_best_sentence(out_sen, sentiment = 0) + "\n")

0 <ATTR_WORDS> all way <CON_START> i ve had this thermometer for num _ num . <START>

['i ve had this thermometer for num _ num years all way .', 'i ve had this thermometer for all num _ num years .', 'i ve had this thermometer for about num _ num years all way .', 'i ve had this thermometer for num _ num years .', 'i ve had this thermometer for over num _ num years all way .']
i ve had this thermometer for num _ num years .
1 <ATTR_WORDS> over num_num inch still <CON_START> will update this review should it meet a similar to my . <START>

['will update this review should it ever meet a similar problem to my over num _ num inch .', 'will update this review should it ever meet a similar problem to my over num inch .', 'will update this review should it ever meet a similar problem to my over inch .', 'will update this review should it meet a similar problem to my original .', 'will update this review should it meet a similar problem to my previous one .']
will update this review should i

### Do Reference 0

In [18]:
# B. during training of G-GST models,
# they take a training example as follows: 
# Training input: <ATTR_WORDS> great excellent <CON_START> The food was and the service was <START> The food was great and the service was excellent . <END>
# 
# instead of target text style they specify the exact attributes to be used 
# to fill in the content string 
# to generate a sentence.


#DONE
"""
with open(os.path.join(output_dir,"reference_0_predictions_with_full_sentence_match_beam_search_bm5.txt") ,"w", encoding='utf-8') as out_fp:
    c = 0
    #with open(os.path.join(output_dir, "./reference_0_content.txt")) as fp:
    with open(os.path.join(output_dir, "./reference_0.txt")) as fp:
        for line in fp:
            out_sen = preditction_with_beam_search(line.strip(), beam_width=5, vocab_length=max(tokenizer.special_tokens.values()) + 1)
            print(c,get_best_sentence(out_sen, sentiment = 1))
            c += 1
            out_fp.write(get_best_sentence(out_sen, sentiment = 1) + "\n")
"""

0 ever since joes has changed hands it ' s just gotten better and better .
1 there is definitely enough space in that part of the venue .
2 so basically tasted fantastic fantastic .
3 she said she ' ll be back and be back for a few minutes .
4 i ca ' t believe how honest this pharmacy is .
5 just great fun and took it to the bill .
6 it is n ' t terrible , but it is n ' t very good either .
7 definitely worth that i could n ' t use my birthday gift !
8 owner , i heard incredible - but i do n ' t know the details .
9 but yes it probably does n ' t suck too bad !
10 we sit down and we got some really modern and professional service .
11 the charge did include miso soup and a small salad .
12 there was dragon i ' m or how everything comes out .
13 we could n ' t wait at the table if we were n ' t ordering dinner .
14 the cash register area was amazing and one was the store front .
15 there chips are ok , but their salsa is really gorgeous .
16 the wine was very average and the food was ev

### Do Reference 1 ( issues here )

In [7]:

with open(os.path.join(output_dir,"reference_1_predictions_with_full_sentence_match_beam_search_bm5.txt") ,"w", encoding='utf-8') as out_fp:
    c = 0
    with open(os.path.join(output_dir, "./reference_1_with_line_9_twice.txt")) as fp:
        for line in fp:
            out_sen = ggst1_pred_with_beam_search(line.strip(), beam_width=5, vocab_length=max(tokenizer.special_tokens.values()) + 1, debug=False)
            b = get_best_sentence(out_sen, sentiment = 0)
            print(c,b)
            c += 1
            out_fp.write(b + "\n")

0 it ' s small yet they do n ' t make you feel at home .
1 i will miss going back and this place misses me !
2 the drinks were affordable and a nightmare to drink .
3 my husband got a ruben sandwich , he got it dry .
4 sadly i signed up for their email and a coupon .
5 no gloves i ' d definitely give them a try .
6 i highly recommend other e & m painting other notice .
7 otherwise a please hire and we will n ' t go again .
8 drinks , and no company being ignored .
9 oh i got my band geek back on now !
10 oh i got my band geek back on now !
11 food steak and tuna were both terrible .
12 everything is so fresh and so tasty !
13 the biscuits and gravy were good .
14 i actually can n ' t wait next year !
15 they were so friendly , mediocre , and reasonably priced .
16 i get pei wei gets boring .
17 disappointed , authentic bagels , tasty cream cheese , tasty cream cheese .
18 you ' ll see why once you get there i do n ' t like it once you get there .
19 everyone that i spoke with was n ' t

In [8]:
# THIS WAS FOR DEBUGGING 
# THE ISSUE SEEMED HARDWARE RELATED, but what I did to fix it was just copy line 10 twice
# and then in the output reference_1_preds file I just copied over the same pred that B-GST gave ( so not 100% identical to bu 999/1000 %)


with open(os.path.join(output_dir,"reference_1_predictions_with_full_sentence_match_beam_search_bm5.txt") ,"w", encoding='utf-8') as out_fp:
    c = 0
    with open(os.path.join(output_dir, "./reference_1.txt")) as fp:
        for line in fp:
            try:
                out_sen = ggst1_pred_with_beam_search(line.strip(), beam_width=5, vocab_length=max(tokenizer.special_tokens.values()) + 1, debug=True)
                b = get_best_sentence(out_sen, sentiment = 0)
                print(out_sen, c,b)
                c += 1
                out_fp.write(b + "\n")
            except Exception as e:
                print(e)
                er = "Error on "+str(c)+" "+line
                print(er)
                c += 1
                out_fp.write(er )
                # The size of tensor a (513) must match the size of tensor b (512) at non-singleton dimension 3
                # Error on 10 <ATTR_WORDS> parts much less <CON_START> i was nervous and she me feel so and . she did not make me feel welcomed <START>

                # CUDA error: device-side assert triggered the rest of the way out
                
                # https://github.com/huggingface/transformers/issues/1805
                #   It's probably because your token embeddings size (vocab size) doesn't match with pre-trained model. 
                #   Do model.resize_token_embeddings(len(tokenizer)) before training. 
                #   Please check #1848 and #1849
                
                # https://github.com/huggingface/transformers/issues/2185
                #   Have you ever read in the Issues section, e.g. #1852, #1848, #1849 and #1805? 
                #  They suggest different solutions for your problem, e.g. changing the input sequence limit to 128.

<ATTR_WORDS> do n't <CON_START> it ' s small yet they you feel at home . it ' s small yet they you feel like a stranger . <START> 30 ['<ATTR_WORDS>', 'do</w>', 'n</w>', "'</w>", 't</w>', '<CON_START>', 'it</w>', "'</w>", 's</w>', 'small</w>', 'yet</w>', 'they</w>', 'you</w>', 'feel</w>', 'at</w>', 'home</w>', '.</w>', 'it</w>', "'</w>", 's</w>', 'small</w>', 'yet</w>', 'they</w>', 'you</w>', 'feel</w>', 'like</w>', 'a</w>', 'stranger</w>', '.</w>', '<START>'] 30 [40478, 587, 247, 256, 241, 40479, 507, 256, 252, 1220, 1340, 600, 512, 1064, 491, 1163, 239, 507, 256, 252, 1220, 1340, 600, 512, 1064, 649, 246, 4602, 239, 40480]
Softmax(dim=-1)
["it ' s small yet they do n ' t treat you like a stranger .", "it ' s small yet they do n ' t make you feel at home .", "it ' s small yet they do n ' t treat you like a stranger .", "it ' s small yet they do n ' t treat you like a stranger .", "it ' s small yet they do n ' t treat you like a stranger ."] 0 it ' s small yet they do n ' t make you fee